# Job Shop Scheduling Sample

## Introduction
Job shop scheduling is a common and important problem in many industries. For example, in the automobile industry manufacturing a car involves many different types of operations which are performed by a number of specialized machines - optimizing the production line to minimize manufacturing time can make for significant cost savings. 

This problem is a superset of the travelling salesman's problem, which we know is NP-Hard. 

The job shop scheduling problem is defined as follows: we have a set of jobs $J_1, J_2, J_3,\dots, J_n$ which have various processing times and need to be processed using a set of machines $m_1, m_2, \dots, m_n$. The goal is to complete all jobs in the shortest time possible. 
Each job consists of a set of operations, and the operations must be performed in the correct order to complete that job. 

Imagine, for example, that we have a to-do list. Each item on the list is a **job** using our new terminology.

Each job in this list consists of a set of operations, and each operation has a processing time. We also have some tools at hand that we can use to complete these jobs (our **machines**).

TODOs:

$$
\begin{array}[t]{ | l | }
    \hline
    \text{Pay electricity bill } \textbf{(Job 0)} \\ \hline
    \text{Log in to site }\textit{(2 mins)}\\ 
    \text{Pay bill }\textit{(1 min)}\\
    \text{Print receipt }\textit{(3 mins)}\\
    \hline
\end{array}
\begin{array}[t]{ | l | }
    \hline
    \text{Plan camping trip } \textbf{(Job 1)} \\ \hline
    \text{Pick campsite }\textit{(2 hours)}\\ 
    \text{Pay online }\textit{(2 mins)}\\
    \text{Print receipt }\textit{(3 mins)}\\
    \hline
\end{array}
\begin{array}[t]{ | l | }
    \hline
    \text{Book dentist appointment } \textbf{(Job 2)} \\ \hline
    \text{Choose time }\textit{(2 mins)}\\ 
    \text{Pay online }\textit{(2 mins)}\\
    \text{Print receipt }\textit{(3 mins)}\\
    \text{Guiltily floss }\textit{(2 mins)}\\
    \hline
\end{array}
$$

The available tools (or **machines**): 

$$
\begin{array}[t]{ | l | }
    \hline
    \text{Tools } \textbf{(machines)} \\ \hline
    \text{Computer } \\ 
    \text{Credit card }\\ 
    \text{Printer }\\
    \text{Dental floss }\\
    \hline
\end{array}
$$

But we have some constraints:
1. Each of the tasks (**operations**) in a todo (**job**) must take place in order. We can't print the receipt before we have made the payment! This is called a **precedence constraint**.
2. Each tool (**machine**) can only do one thing at a time. We can't simultaneously print two receipts unless we invest in multiple printers. This is the **no overlap constraint**.
3. We start an operation only once, and once started it must be completed before we do anything else. We don't tolerate procrastination in this shop! This is called the **operation once constraint**.

## Azure Quantum Setup

Before we get started with formulating our problem, we need to import some Python modules and set up our Azure Quantum `Workspace`. You will need to enter your Azure Quantum workspace details in the cell below before you run it:

In [1]:
from typing import List
from azure.quantum.optimization import Term
from azure.quantum import Workspace

workspace = Workspace (
    subscription_id = "<subscription_id>", # Add your subscription_id
    resource_group = "<resource_group>",   # Add your resource_group
    name = "<workspace_name>"              # Add your workspace name
    )

workspace.login()

ValueError: Failed to get tenant authority for subscription '<subscription_id>'. Make sure the subscription id is correct.

## Problem Formulation

Now that we have set up our development environment, we can start to formulate our problem.

The first step is to take the constraints we identified above and formulate them as mathematical equations that we can work with. 

Let's first introduce some notation because we are lazy and also want to avoid carpal tunnel syndrome.

We’ll stick with our previous example of the todo list for now:

$$
\begin{array}[t]{ | cl | }
    \hline
    J_{0} \text{:} & \text{Pay electricity bill} \\ \hline
    O_{0} & \text{Log in to site}\\ 
    O_{1} & \text{Pay bill}\\
    O_{2} & \text{Print receipt}\\
    \hline
\end{array}
\begin{array}[t]{ | cl | }
    \hline
    J_{1} \text{:} & \text{Plan camping trip} \\ \hline
    O_{3} & \text{Pick campsite}\\ 
    O_{4} & \text{Pay online}\\
    O_{5} & \text{Print receipt}\\
    \hline
\end{array}
\begin{array}[t]{ | cl | }
    \hline
    J_{2} \text{:} & \text{Book dentist appointment} \\ \hline
    O_{6} & \text{Choose time}\\ 
    O_{7} & \text{Pay online}\\
    O_{8} & \text{Print receipt}\\
    O_{9} & \text{Guiltily floss}\\
    \hline
\end{array}
$$

Above, you can see that we’ve labelled our jobs as $J$ and given them index numbers $0$, $1$ and $2$, to represent each of the three todos we had. We have also defined the operations that make up each job, represented by the letter $O$.

To make it easier to code up later, we have chosen to identify all operations with a continuous index number rather than (for example) starting from $0$ for each job - this allows us to keep track of operations by this ID number in the code and schedule them according to our constraints and machine availability. We can tie the operations back to their jobs later on using a reference. 

Below, you see how these definitions combine to give us a mathematical formulation for our jobs:

$$
\begin{align}
J_{0} &= \{O_{0}, O_{1}, O_{2}\} \\
J_{1} &= \{O_{3}, O_{4}, O_{5}\} \\
J_{2} &= \{O_{6}, O_{7}, O_{8}, O_{9}\} \\
\end{align}
$$

**More generally:**

$$
\begin{align}
J_{0} &= \{O_{0}, O_{1}, \ldots , O_{k_{0}-1}\} \text{, where } k_{0} = x_{0} \text{, the number of operations in job } J_{0}\\
\\
J_{1} &= \{O_{k_{0}}, O_{k_{0}+1}, \ldots , O_{k_{1}-1}\} \text{, where } k_{1} = x_{0} + x_{1} \text{, the number of operations in jobs } J_{0} \text{ and } J_{1} \text{ combined}\\
\\
&\vdots \\
\\
J_{n-1} &= \{O_{k_{n-2}}, O_{k_{n-2}+1}, \ldots , O_{k_{n-1}-1}\} \text{, where } k_{n-1} = \text{ the total number of operations across all jobs }\\
\end{align}
$$ 

The next piece of notation we will need is a binary variable which we will call $x_{i, t}$

We will use this variable to represent whether an operation starts at time $t$ or not:

$$
\begin{align}
\text{If } x_{i,t} &= 1, \text{ } O_i\text{ starts at time } \textit{t} \\
\text{If } x_{i,t} &= 0, \text{ } O_i\text{ does not start at time } \textit{t} \\
\end{align}
$$

> This makes this a binary optimisation – more generally, this is called a polynomial unconstrained binary optimisation (or PUBO) problem. You may also see these PUBO problems referred to as Higher Order Binomial Optimization (HOBO) problems - these terms both refer to the same thing.

We use $t$ to represent our simulation time. It goes from time $0$ through to $T$ in integer steps. $T$ is the longest we are happy for the whole set of jobs to take in total (i.e. the max simulation time):

$$0 \leq t < T$$

Lastly, we define $p_{i}$ to be the processing time for operation $i$ - the amount of time it takes for operation $i$ ($O_{i}$) to complete:

$$\text{If } O_{i} \text{ starts at time } \textit{t} \text{, it will finish at time } t + p_{i}$$
$$\text{If } O_{i+1} \text{ starts at time } \textit{s} \text{, it will finish at time } s + p_{i+1}$$

Now that we have defined our terms, we can move on to formulating the problem.

We want to be able to represent our constraints mathematically. We’ll do this using a penalty model - every time our optimizer suggests a solution that violates one or more constraints, we need to give that solution a penalty:

$$
\begin{array}{ | l | l | }
    \hline
    \textbf{Constraint} & \textbf{Penalty Condition} \\ \hline
    \textbf{Precedence constraint} & \text{Assign penalty every time } O_{i+1} \text{ starts before}\\
    \text{Operations in a job must take place in order} & O_{i} \text{ has finished (i.e. they start out of order)} \\ \hline
    \textbf{No overlap constraint} & \text{Assign penalty every time two operations on a single}\\
    \text{Machines can only do one thing at a time} & \text{machine are scheduled to run at the same time} \\ \hline
    \hline
    \textbf{Operation once constraint} & \text{Assign penalty if an operation isn't scheduled within} \\
    \text{Each operation is started once and only once} & \text{the allowed time } T \text{, or if scheduled more than once} \\
    \text{Once an operation starts, it runs to completion} & \textit{Assumption: if an operation starts, it completes} \\ \hline
    \hline
\end{array}
$$


###  Expressing a Cost Function Using the Azure Quantum Optimization SDK

> As we will see during our exploration of the cost function and its constituent penalty terms below, the overall cost function is quadratic (because the highest order polynomial term we have is squared). This makes this problem a **Quadratic Unconstrained Binary Optimization (QUBO)** problem, which is a specific subset of **Polynomial Unconstrained Binary Optimization (PUBO)** problems (which allow for higher-order polynomial terms than quadratic). Fortunately for us, the Azure Quantum Optimization service is set up to accept PUBO (and Ising) problems, which means we don't need to modify our representation to fit the solver.

As introduced above, the binary variable we are optimizing for here is $x_{i,t}$, which can take a value of either 0 or 1, depending on if the operation $i$ starts at time $t$ or not:

$$
\begin{align}
\text{If } x_{i,t} &= 1, \text{ } O_i\text{ starts at time } \textit{t} \\
\text{If } x_{i,t} &= 0, \text{ } O_i\text{ does not start at time } \textit{t} \\
\end{align}
$$

For $t = 0 \rightarrow t < T$ for every operation, we define an index $x_{i + t}$, which means that every operation in a job contributes to $T$ indices. 

The operation starts at the value of $t$ for which $x_{i + t}$ equals 1.

In the next sections, we will construct mathematical representations of the penalty terms and use these to build our code. The code outputs an array of `Term`s, where each term is an object that looks like:

```python
(c: float, indices: []) # Constant terms e.g. +1
(c: float, indices: [int]) # Linear terms e.g. x
(c: float, indices: [int, int]) # Quadratic terms e.g. x^2
```

The `c` element represents the weight (coefficient) for each term, and the `indices` array represents the indices $i + t$ of the $x_{i+t}$ values. 

If we had higher order terms (e.g. cubed etc.), we would just add more elements to the indices array, like so:

```python
(c: float, indices: [int, int, int, ...])
```

We’ll now explore how to formulate each of these constraints mathematically, and how this translates to code.

> **Note 1:** For this sample, we do not add penalties for operations that overrun the total simulation time $T$ - we assume that as long as an operation is started before the end of the simulation, it runs to completion.

> **Note 2:** To simplify the code, we have set all jobs to have the same number of operations.

### Precedence Constraint

$$
\begin{array}{ | l | l | }
    \hline
    \textbf{Constraint} & \textbf{Penalty Condition} \\ \hline
    \textbf{Precedence constraint} & \text{Assign penalty every time } O_{i+1} \text{ starts before}\\
    \text{Operations in a job must take place in order} & O_{i} \text{ has finished (i.e. they start out of order)} \\ \hline
\end{array}
$$

#### Worked Example

We will take job 1 ($J_{1}$) as an example: 

$$
\begin{array}{ | cl | }
    \hline
    J_{1} \text{:} & \text{Plan camping trip} \\ \hline
    O_{3} & \text{Pick campsite}\\ 
    O_{4} & \text{Pay online}\\
    O_{5} & \text{Print receipt}\\
    \hline
\end{array}
$$



Let's formulate the penalty conditions for $O_{3}$ and $O_{4}$: we want to add a penalty if $O_{4}$ starts before $O_{3}$ finishes. First, we'll define our terms and set some of their values:

$$\text{Total simulation time } T = 4$$
$$O_{3} \text{ processing time: } p_{3} = 2$$
$$O_{3} \text{ starts at time } \textit{t} \text{, and finishes at time } t+p_{3}$$

$$O_{3} \text{ starts at any time } 0 \leq t < T $$
$$O_{4} \text{ can start at time } s \geq t + p_{3} $$

$O_{3}$’s finishing time is given by adding its processing time $p_{3}$ (which we’ve set to be 2) to its start time $t$. You can see the start and end times for $O_{3}$ in the table below:

$$
\begin{array}{ | c | c | c | c | }
    \hline
    t & t+p_{3}\\ \hline
    0 & 2 \\ \hline
    1 & 3 \\ \hline
    2 & 4 \\ \hline
    \hline
\end{array}
$$

To avoid violating this constraint, the start time of $O_{4}$ (denoted by $s$) must be greater than or equal to the end time of $O_{3}$, like we see in the next column:

$$
\begin{array}{ | c | c | c | }
    \hline
    t & t+p_{3} & s \geq t+p_{3}\\ \hline
    0 & 2 & 2, 3, 4 \\ \hline
    1 & 3 & 3, 4 \\ \hline
    2 & 4 & 4 \\ \hline
    & \text{Valid?} & \checkmark \\
    \hline
\end{array}
$$

The $\checkmark$ means that any $s$ value in this column is valid, as it doesn't violate the precedence constraint.

Conversely, if $s$ is less than $t + p_{3}$ (i.e. $O_{4}$ starts before $O_{3}$ finishes), we need to add a penalty. Invalid $s$ values for this example are shown in the rightmost column:

$$
\begin{array}{ | c | c | c | c | }
    \hline
    t & t+p_{3} & s \geq t+p_{3} & s < t+p_{3}\\ \hline
    0 & 2 & 2, 3, 4 & 0, 1 \\ \hline
    1 & 3 & 3, 4 & 0, 1, 2 \\ \hline
    2 & 4 & 4 & 0, 1, 2, 3 \\ \hline
     & \text{Valid?} & \checkmark & \text{X} \\
    \hline
\end{array}
$$

We have used an $\text{X}$ mark to denote that any $s$ value in the last column is invalid, as it violates the precedence constraint.

#### Penalty Formulation

We formulate this as a penalty by counting every time consecutive operations $O_{i}$ and $O_{i + 1}$ in a job take place out of order. 
  
As we saw above: for an operation $O_{i}$, if the start time of $O_{i + 1}$ (denoted by $s$) is less than the start time of $O_{i}$ (denoted by $t$) plus its processing time $p_{i}$, then that counts as a penalty. Mathematically, this penalty condition looks like: $s < t + p_{i}$ 

We sum that penalty over all the operations of a job ($J_{n}$) for all the jobs:
$$f(x) = \mathop{\sum_{k_{n-1} \leq i < k_n}}_{t+p_i>s}x_{i,t}\cdot x_{i+1,s} \text{ for each job } \textit{n} \text{,}$$

Let's break that down:

- $k_{n-1} \leq i < k_{n}$

  This means we sum over all operations for a single job.
<br/>

- $s < t + p_{i}$

  This is our penalty condition - any operation that satisfies this condition is in violation of the precedence constraint.
<br/>

- $x_{i, t}\cdot x_{i+1, s}$
  
  This represents the table we built out in the example, where $t$ is allowed to vary from $0 \rightarrow T$ and we assign a penalty whenever the constraint is violated (when $s < t + p_{i}$).
  
  This translates to a nested `for` loop: the outer loop has limits $0 \leq t < T$ and the inner loop has limits $0 \leq s < t + p_{i}$
<br/>

#### Code

Using our mathematical formulation and the breakdown above, we can now translate this constraint function to code:

In [3]:
# T: Time allowed to complete all operations
# n: Total number of jobs
# o: Number of operations per job
# p: List of job processing times
# c: Relative weight of this constraint (the coefficient)
def precedence_constraint(n: int, o:int, T:int, c:float, p:List[int]):
    terms = []
    j = 0

    # Loop through all jobs:
    while(j < n):
        # Loop through all operations in this job:
        for i in range(j, j + o - 1):
            # Loop through simulation time:
            for t in range(0, T):
                # Loop over times that would violate the constraint:
                for s in range(0, t + p[i]):
                    # Assign penalty
                    terms.append(Term(c = c*1, indices = [i*T+t, (i+1)*T+s]))
        j = j + o
    return terms

>Note: This nested loop structure is probably not the most efficient way to do this but it is the most direct comparison to the mathematical formulation.

### Operation Once Constraint

$$
\begin{array}{ | l | l | }
    \hline
    \textbf{Constraint} & \textbf{Penalty Condition} \\ \hline
    \textbf{Operation once constraint} & \text{Assign penalty if an operation isn't scheduled within} \\
    \text{Each operation is started once and only once} & \text{the allowed time } T \text{, or if scheduled more than once} \\
    \text{Once an operation starts, it runs to completion} & \textit{Assumption: if an operation starts, it completes} \\ \hline
    \hline
\end{array}
$$

#### Worked Example

We will again take job 1 ($J_{1}$) as an example:

$$
\begin{array}{ | cl | }
    \hline
    J_{1} \text{:} & \text{Plan camping trip} \\ \hline
    O_{3} & \text{Pick campsite}\\ 
    O_{4} & \text{Pay online}\\
    O_{5} & \text{Print receipt}\\
    \hline
\end{array}
$$

Recall our variable $x_{i,t}$:

$$
\begin{align}
\text{If } x_{i,t} &= 1, \text{ } O_i\text{ starts at time } \textit{t} \\
\text{If } x_{i,t} &= 0, \text{ } O_i\text{ does not start at time } \textit{t} \\
\end{align}
$$

According to this constraint, $x_{i,t}$ for a specific operation should equal 1 **once and only once** during the entire simulation from $t = 0 \rightarrow T$ (because it should start once and only once during the allowed time).

So in this case, we need to assign a penalty if the sum of $x_{i,t}$ for each operation across the full simulation time doesn’t equal exactly 1.

Let’s take $O_{3}$ as an example again:

$$
\begin{array}{ | c | c | }
    \hline
    t & \text{  } x_{3,t} \text{  } \\ \hline
    0 & 0 \\ \hline
    1 & 1 \\ \hline
    2 & 0 \\ \hline
     & \\ \hline
    \sum_t {x_{3,t}} = & 1 \\ \hline
    \text{Valid?} & \checkmark \\
    \hline
\end{array}
$$

In the right hand column, we see that $O_{3}$ starts at time 1 and no other time ($x_{3,t} = 1$ at time $t = 1$ and is $0$ otherwise). The sum of $x_{i,t}$ values over all $t$ for this example is therefore 1, which is what we expect! This is therefore a valid solution.

In the example below, we see an instance where $O_{3}$ is scheduled more than once ($x_{3,t} = 1$ more than once), in violation of the constraint:

$$
\begin{array}{ | c | c | }
    \hline
    t & \text{  } x_{3,t} \text{  } \\ \hline
    0 & 0 \\ \hline
    1 & 1 \\ \hline
    2 & 1 \\ \hline
     &  \\ \hline
    \sum_t {x_{3,t}} = & 2 \\ \hline
    \text{Valid?} & \text{X} \\
    \hline
\end{array}
$$

We can see from the above that $O_{3}$ has been scheduled to start at both time 1 and time 2, so our sum of $x_{i,t}$ values over all $t$ is now greater than 1. This violates our constraint and thus we must apply a penalty.

In the last example, we see an instance where $O_{3}$ has not been scheduled at all:

$$
\begin{array}{ | c | c | }
    \hline
    t & \text{  } x_{3,t} \text{  } \\ \hline
    0 & 0 \\ \hline
    1 & 0 \\ \hline
    2 & 0 \\ \hline
     & \\ \hline
    \sum_t {x_{3,t}} = & 0 \\ \hline
    \text{Valid?} & \text{X} \\
    \hline
\end{array}
$$

In this example, none of the $x_{3,t}$ values equal 1 for any time in the simulation, meaning the operation is never scheduled. This means that the sum of $x_{3,t}$ values over all $t$ is 0 - the constraint is once again violated and we must allocate a penalty.

In summary:

$$
\begin{array}{ | c | c | c | c | }
    \hline
    t & \text{  } x_{3,t} \text{  } & \text{  } x_{3,t} \text{  } & \text{  } x_{3,t} \text{  } \\ \hline
    0 & 0 & 0 & 0 \\ \hline
    1 & 1 & 1 & 0 \\ \hline
    2 & 0 & 1 & 0 \\ \hline
     & & & \\ \hline
    \sum_t {x_{3,t}} = & 1 & 2 & 0 \\ \hline
    \text{Valid?} & \checkmark & \text{X} & \text{X} \\
    \hline
\end{array}
$$

Now we understand when to assign penalties, let's formulate the constraint mathematically.

#### Penalty Formulation

As seen previously, we want to assign a penalty whenever the sum of $x_{i,t}$ values across all possible $t$ values is not equal to 1. This is how we represent that mathematically:

$$g(x) = \sum_{i} \left(\left(\sum_{0\leq t < T} x_{i,t}\right) - 1\right)^2$$

Let's break that down:

- $\left(\sum_{0\leq t < T} x_{i,t}\right) - 1$

  As we saw in the sum row of the tables in the worked example, $\sum_{0\leq t < T} x_{i,t}$ should always equal exactly 1 (meaning that an operation must be scheduled **once and only once** during the simulation). This means that $\left(\sum_{0\leq t < T} x_{i,t}\right) - 1$ should always give 0. This means there is no penalty assigned when the constraint is not violated.
  
  In the case where $\sum_{0\leq t < T} x_{i,t} > 1$ (i.e. an operation is scheduled to start more than once, like in the second example above), we now have a positive, non-zero penalty term as $\left(\sum_{0\leq t < T} x_{i,t}\right) - 1 > 0$ now.
  
  In the case where $\sum_{0\leq t < T} x_{i,t} = 0$ (i.e. an operation is never scheduled to start, like in the last example above), we now have a $-1$ penalty term as $\left(\sum_{0\leq t < T} x_{i,t}\right) - 1 = 0 - 1 = -1$

<br/>

- $\left(\sum\dots\right)^2$

  Because our penalty terms must always be positive (otherwise we would be *reducing* the penalty when an operation isn't scheduled), we must square the result of $\left(\sum_{0\leq t < T} x_{i,t}\right) - 1$
  
  This ensures that the penalty term is always positive (as $(-1)^2 = 1$).
  
<br/>

- $\sum_{i} \left((\dots)^2\right)$

  Lastly, we must sum all penalties accumulated across all operations $O_{i}$ from all jobs.
  
To translate this constraint to code form, we are going to need to expand the quadratic equation in the sum.

To do this, we'll once again take $O_{3}$ as an example. We will set $T = 2$ so our $t$ values will be 0 and 1. The first step will be to substitute in our values:

$$
\begin{align}
\sum_{i} \left(\left(\sum_{0\leq t < T} x_{i,t}\right) - 1\right)^2 &= \left(x_{3,0} + x_{3,1} - 1\right)^2 
\end{align}
$$

For simplicity, we will rename these $x_{3,t}$ variables as follows: 

$$
\begin{align}
x_{3,0} &= x \\
x_{3,1} &= y
\end{align}
$$

Substituting these values in, we now have the following:

$$
\begin{align}
\sum_{i} \left(\left(\sum_{0\leq t < T} x_{i,t}\right) - 1\right)^2 &= \left(x_{3,0} + x_{3,1} - 1\right)^2 \\
&=\left(x + y - 1\right)^2
\end{align}
$$

Next, we expand out the bracket and multiply each term in the first bracket with all terms in the other bracket:

$$
\begin{align}
\sum_{i} \left(\left(\sum_{0\leq t < T} x_{i,t}\right) - 1\right)^2 &= \left(x_{3,0} + x_{3,1} - 1\right)^2 \\
&= \left(x + y - 1\right)^2 \\
&= (x + y - 1)\cdot(x + y - 1) \\
&= x^2 + y^2 + 2xy - 2x - 2y + 1
\end{align}
$$

Of course, if $T$ was larger, we would have more terms. The form of the equation would be the same however: still quadratic. 

#### Code

We can now use this expanded version of the penalty function to build our penalty terms in code:

In [4]:
# T: Time allowed to complete all operations
# n: Total number of jobs
# o: Number of operations per job
# c: Relative weight of this constraint (the coefficient)

# Penalty function is of form: x^2 + y^2 + 2xy - 2x - 2y + 1
def operation_once_constraint(n: int, o: int, T:int, c:float):
    terms = []
    
    # x^2 + y^2 + 2xy - 2x - 2y parts of the constraint function
    # Loop through all operations
    for i in range(n*o):
        for t in range(T):
            # x^2 + y^2 terms
            terms.append(Term(c=c*1, indices=[i*T+t, i*T+t]))

            # - 2x - 2y terms
            terms.append(Term(c=c*-2, indices=[i*T+t]))
            
            # + 2xy term
            # Loop through all other start times for the same job
            # to get the cross terms
            for s in range(t+1, T):
                terms.append(Term(c=c*2, indices=[i*T+t, i*T+s]))
    
    # + 1 term
    terms.append(Term(c=c*1, indices=[]))
    
    return terms

### No Overlap Constraint

$$
\begin{array}{ | l | l | }
    \hline
    \textbf{Constraint} & \textbf{Penalty Condition} \\ \hline
    \textbf{No overlap constraint} & \text{Assign penalty every time two operations on a single}\\
    \text{Machines can only do one thing at a time} & \text{machine are scheduled to run at the same time} \\ \hline
    \hline
\end{array}
$$

#### Worked Example

For this final constraint, we will again use $J_{1}$ as an example:

$$
\begin{array}{ | cl | }
    \hline
    J_{1} \text{:} & \text{Plan camping trip} \\ \hline
    O_{3} & \text{Pick campsite}\\ 
    O_{4} & \text{Pay online}\\
    O_{5} & \text{Print receipt}\\
    \hline
\end{array}
$$

Recall once more our variable $x_{i,t}$:

$$
\begin{align}
\text{If } x_{i,t} &= 1, \text{ } O_i\text{ starts at time } \textit{t} \\
\text{If } x_{i,t} &= 0, \text{ } O_i\text{ does not start at time } \textit{t} \\
\end{align}
$$

Let's say that $O_{3}$ and $O_{4}$ must be completed using the same machine. To avoid violating the no overlap constraint, we must ensure that $O_{3}$ and $O_{4}$ begin at different times: i.e. $x_{3,t}$ and $x_{4,t}$ must not equal 1 at the same time. 

One example of a valid configuration is shown below:

$$
\begin{array}{ | c | c | c | c | }
    \hline
    \text{  } t \text{  } & \text{  } x_{3,t} \text{  } & \text{  } x_{4,t} \text{  } & x_{3,t} \cdot x_{4,t}\\ \hline
    0 & 1 & 0 & 0\\ \hline
    1 & 0 & 1 & 0\\ \hline
    2 & 0 & 0 & 0\\ \hline
      &   & \sum_{t} x_{3,t} \cdot x_{4,t} = & 0 \\
    \hline
\end{array}
$$

As we can see, when we compare $x_{i,t}$ values pairwise at each time in the simulation, their product always equals 0.

Below, we see a configuration that violates the constraint:

$$
\begin{array}{ | c | c | c | c | }
    \hline
    \text{  } t \text{  } & \text{  } x_{3,t} \text{  } & \text{  } x_{4,t} \text{  } & x_{3,t} \cdot x_{4,t}\\ \hline
    0 & 0 & 0 & 0\\ \hline
    1 & 1 & 1 & 1\\ \hline
    2 & 0 & 0 & 0\\ \hline
      &   & \sum_{t} x_{3,t} \cdot x_{4,t} = & 1 \\
    \hline
\end{array}
$$

In this instance, $O_{3}$ and $O_{4}$ are both scheduled to start at $t = 1$ and given they require the same machine, this means that the constraint has been violated. The pairwise product of $x_{i,t}$ values is therefore no longer always equal to 0, as for $t = 1$ we have: $x_{3,1} \cdot x_{4,1} = 1$ 

We can now use this knowledge to mathematically formulate our constraint.

#### Penalty Formulation

As we saw from the tables in the worked example, for our configuration to be valid, the sum of pairwise products of $x_{i,t}$ values for a machine $m$ at any time $t$ must equal 0. This gives us our penalty function:

$$h(x) = \sum_{i,t,k,s} x_{i,t}\cdot x_{k,s} = 0 \text{ for each machine } \textit{m}$$

Let's break that down:

- $\sum_{i,t,k,s}$

  For operation $i$ starting at time $t$, and operation $k$ starting at time $s$, we will sum over all possible start times $0 \leq t < T$ and $0 \leq s < T$. This indicates to us the need for another nested `for` loop, like we saw for the precedence constraint.
  
  For this summation, $i \neq k$ (i.e. we are always scheduling two different operations).
  
  For two operations happening on a single machine, $t \neq s$ or the constraint has been violated. If $t = s$ for the operations, they have been scheduled to start on the same machine at the same time, which isn't possible.
  
  <br/>
  <br/>
  
- $x_{i,t}\cdot x_{k,s}$

  This is the product we saw explicitly calculated in the rightmost columns of the tables from the worked example. If two different operations $i$ and $k$ start at the same time (i.e. $t = s$), this product will equal 1. Otherwise, it will equal 0.
  
  <br/>
  <br/>
  
- $\sum(\dots) = 0 \text{ for each machine } \textit{m}$

  We perform this sum for each machine $m$ independently.
  
  If all $x_{i,t} \cdot x_{k,s}$ products in the summation equal 0, the total sum comes to 0. This means no operations have been scheduled to start at the same time on this machine and thus the constraint has not been violated. You can see an example of this in the bottom row of the first table from the worked example, above.
  
  If any of the $x_{i,t} \cdot x_{k,s}$ products in the summation equal 1, this means that $t = s$ for those operations and therefore two operations have been scheduled to start at the same time on the same machine. The sum now returns a value greater than 1, which gives us a penalty every time the constraint is violated. You can see an example of this in the bottom row of the second table from the worked example.
  
#### Code

Using the above, we can transform our final penalty function into code which will generate the terms needed by the solver:

In [5]:
# T: Time allowed to complete all operations
# n: Total number of jobs
# p: List of job processing times
# c: Relative weight of this constraint (the coefficient)
# ops_machine_map: Mapping of operations to machines, e.g.:
#     ops_machines_map = [
#         [0,1],          # Operations 0 & 1 assigned to machine 0
#         [2,3]           # Operations 2 & 3 assigned to machine 1
#     ]
def no_overlap_constraint(n: int, T:int, c:float, p: List[int], ops_machines_map: List[List[int]]):
    terms = []
    
    # For each machine
    for m in range(len(ops_machines_map)):
        # Get operations assigned to this machine
        ops = ops_machines_map[m]

        # Loop over each operation i requiring this machine
        for i in range(len(ops)):
            # Loop over each other operation k requiring this machine 
            for k in range(len(ops)):
                # Loop over simulation time
                for t in range(T):
                    # When i != k (i.e. when we are scheduling two different operations)
                    if ops[i] != ops[k]:
                        # t = s i.e. two operations scheduled to start at the same time on the same machine
                        terms.append(Term(c = c*1, indices = [ops[i]*T+t, ops[k]*T+t]))
                    
                    # When i < k, add penalty when O_k starts before O_i has finished
                    if ops[i] < ops[k]:
                        for s in range(0, t + p[ops[i]] - 1):
                            terms.append(Term(c = c*1, indices = [ops[i]*T+t, ops[k]*T+s]))   
    return terms

### Putting it all Together

Now that we have penalty terms to represent all of our constraints, we can finally assemble our cost function, $H(x)$!

As a reminder, here are our penalty terms:

- **Precedence constraint**:

$$f(x) = \mathop{\sum_{k_{n-1} \leq i < k_n}}_{t+p_i>s}x_{i,t}\cdot x_{i+1,s} \text{ for each job } \textit{n}$$

  <br/>
  <br/>
  
- **Operation once constraint**:

$$g(x) = \sum_{i} \left(\left(\sum_{0\leq t < T} x_{i,t}\right) - 1\right)^2$$

  <br/>
  <br/>
  
- **No overlap constraint**:

$$h(x) = \sum_{i,t,k,s} x_{i,t}\cdot x_{k,s} = 0 \text{ for each machine } \textit{m}$$

  <br/>
  <br/>
  
Combining the above turns out to be simpler than you might expect - all we need to do is assign each penalty term a weight and add all the weighted terms together, like so:

$$H(x) = \alpha \cdot f(x) + \beta \cdot g(x) + \gamma \cdot h(x) $$

$$\text{where }\alpha, \beta \text{ and } \gamma \text{ represent the different weights we assign to the penalities.}$$

The weights represent how important each penalty function is, relative to all the others. 

> Along with modifying your cost function (i.e. how you represent the penalties), tuning these weights will define how much success you will have solving your optimization problem. There are many ways to represent each optimization problem's penalty functions and many ways to manipulate their relative weights, so this may require some experimentation before you see success.

#### Code

The code snippet below shows us setting problem parameters, assigning weight values and assembling penalty terms by  summing our penalty functions, as was just demonstrated mathematically. These terms represent our cost function and we will submit them to the solver.

In [6]:
# Set problem parameters
n = 3;                  # Number of jobs
o = 2                   # Number of operations per job
P = [2,1,2,2,1,2];      # Processing time for each operation
T = 5;                  # Time we will allow for all jobs to complete

# Six jobs, two machines
ops_machines_map = [
    [0,1,4,5],          # Machine 0 runs jobs 0, 1, 4 and 5
    [2,3]               # Machine 1 runs jobs 2 and 3
]

# Generate terms to submit to solver using penalty functions defined above
# Assign penalty term weights:
alpha = 0.6 # Precedence constraint
beta = 0.2 # Operation once constraint
gamma = 0.2  # No overlap constraint

# Build terms:
terms = []
c1 = precedence_constraint(n, o, T, alpha, P)
c2 = operation_once_constraint(n, o, T, beta)
c3 = no_overlap_constraint(n, T, gamma, P, ops_machines_map)

# Combine terms:
terms = c1 + c2 + c3

### Submit Problem to Azure Quantum

This code submits our terms to the Azure Quantum `SimulatedAnnealing` solver. We could also have used the same problem definition with any of the other Azure Quantum solvers available (e.g. `ParallelTempering`).

The job is run synchronously in this instance, however this could also be submitted asynchronously as shown in the next subsection.

In [7]:
from azure.quantum.optimization import Problem, ProblemType, SimulatedAnnealing

# Problem type is PUBO in this instance. We could also have chosen to represent our problem in Ising form.
problem = Problem(name="Job shop sample", problem_type=ProblemType.pubo, terms=terms)

# Provide details of our workspace, created at the beginning of this tutorial
solver = SimulatedAnnealing(workspace, timeout = 100) # Timeout in seconds

# Run job synchronously
result = solver.optimize(problem)
config = result['configuration']

print(config)

........
{'0': 1, '5': 0, '6': 0, '1': 0, '7': 1, '2': 0, '8': 0, '3': 0, '9': 0, '4': 0, '10': 1, '15': 0, '16': 0, '11': 0, '17': 1, '12': 0, '18': 0, '13': 0, '19': 0, '14': 0, '20': 0, '21': 0, '22': 0, '23': 1, '24': 0, '25': 0, '26': 0, '27': 0, '28': 0, '29': 1}


### Run Job Asynchronously

Alternatively, a job can be run asynchronously, as shown below:

```python
# Submit problem to solver
job = solver.submit(problem)
print(job.id)

# Get job status
job.refresh()
print(job.details.status)

# Get results
result = job.get_results()
config = result['configuration']
print(config)
```

### Map Variables to Operations

This code snippet contains several helper functions which are used to parse the results returned from the solver and print them to screen in a user-friendly format.

In [8]:
# Create array from returned config dict
def create_op_array(config):
    variables = []
    for key, val in config.items():
        variables.insert(int(key), val)
    return variables

# Print problem details e.g. operation runtimes and machine assignments
def print_problem_details():
    job = 0
    jobs = []
    ops = []
    machines = []
    
    for i in range(o * n):
        jobs.append(job)
        ops.append(i)
        
        if (i + 1) % o == 0:
            job += 1        
    for i in range(len(ops_machines_map)):
        for j in range(len(ops_machines_map[i])):
            machines.insert(ops_machines_map[i][j], i)
    
    print(f"           Job ID: {jobs}")
    print(f"     Operation ID: {ops}")
    print(f"Operation runtime: {P}")
    print(f" Assigned machine: {machines}")
    print()
    
# Split array into rows representing the rows of our operation matrix
def split_array(T:int, variables:List[int]):
    ops = []
    i = 0
    while i < len(variables):
        x = variables[i:i+T]
        ops.append(x)
        i = i + T
    return ops

# Print final output matrix
def print_matrix(matrix):
    labels = "    t:"
    for t in range(0, T):
        labels += f" {t}"
    print(labels)
    
    idx = 0
    for row in matrix:
        print("x_" + str(idx) + ",t: ", end="")
        print(' '.join(map(str,row)))
        idx += 1
    print()

# Group operations into jobs
def get_jobs(variables:List[List[int]]):
    i = 0; 
    jobs = []
    while i < o * n:
        x = []
        for j in range (i, i + o):
            try :
                index = variables[j].index(1)
            except ValueError:
                index = -1
            x.append(index)
        jobs.append(x)
        i += o
    return jobs

### Results

Finally, we take the config returned by the solver and read out the results.

In [9]:
#Produce 1D array of x_i,t = 0, 1 representing when each operation starts
op_array = create_op_array(config) 

# Print config details:
print(f"Config dict:\n{config}\n")
print(f"Config array:\n{op_array}\n")

# Print problem setup
print_problem_details()

# Print final operation matrix, using the returned config
print("Operation matrix:")
matrix = split_array(T, op_array) 
print_matrix(matrix)

# Find where each operation starts (when x_i,t = 1) and return the start time
print("Operation start times (grouped into jobs):")
solution = get_jobs(matrix) 
print(solution)

Config dict:
{'0': 1, '5': 0, '6': 0, '1': 0, '7': 1, '2': 0, '8': 0, '3': 0, '9': 0, '4': 0, '10': 1, '15': 0, '16': 0, '11': 0, '17': 1, '12': 0, '18': 0, '13': 0, '19': 0, '14': 0, '20': 0, '21': 0, '22': 0, '23': 1, '24': 0, '25': 0, '26': 0, '27': 0, '28': 0, '29': 1}

Config array:
[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]

           Job ID: [0, 0, 1, 1, 2, 2]
     Operation ID: [0, 1, 2, 3, 4, 5]
Operation runtime: [2, 1, 2, 2, 1, 2]
 Assigned machine: [0, 0, 1, 1, 0, 0]

Operation matrix:
    t: 0 1 2 3 4
x_0,t: 1 0 0 0 0
x_1,t: 0 0 1 0 0
x_2,t: 1 0 0 0 0
x_3,t: 0 0 1 0 0
x_4,t: 0 0 0 1 0
x_5,t: 0 0 0 0 1

Operation start times (grouped into jobs):
[[0, 2], [0, 2], [3, 4]]
